In [29]:
import re
from collections import defaultdict
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 10000)
pd.options.display.max_rows = 5000
import warnings
warnings.filterwarnings("ignore")
import sys

#from similarityfunctionGuise import *

from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve
import pickle

import os
cwd = os.getcwd()
print("current directory", cwd)

current directory C:\Users\garvarun\Desktop\kpi2


In [30]:
actual_df = pd.read_excel(r"Aflac Performance Dashboard -  EOM Feb 2023.xlsb")
actual_df = actual_df[["session_id", "agent", "evaluated_by", "section_2_answer_5", "section_2_answer_5_score", 
                       "section_2_question_5_comment"]]

actual_df.loc[actual_df["section_2_answer_5"].isna(), "Actual"] = -1
actual_df.loc[actual_df["section_2_answer_5"] == "Yes", "Actual"] = 1
actual_df.loc[actual_df["section_2_answer_5"] == "No", "Actual"] = 0

print(actual_df.shape[0])
print(len(actual_df["session_id"].unique()))
print(actual_df["Actual"].value_counts())

8855
8827
 1.0    7050
-1.0    1542
 0.0     263
Name: Actual, dtype: int64


In [31]:
actual_df_2 = pd.read_excel(r"agreement.xlsx")


In [32]:
actual_df_1 = pd.read_excel(r"aflac dashboard.xlsx")


In [40]:
file_path = "aflac_actorchat_5246.pkl"

# Read the pickled file and store its contents in the variable 'data_raw'
with open(file_path, 'rb') as file:
    data_raw = pickle.load(file)

In [6]:
data_raw

,session_id,start_date,end_date,call_type,agent_id,agent_name,timestamp,chat_text,entity_type,sentiment_score,...,num_transfers,overlap_time,max_silence,sentiment,num_holds,last_speaker,aperture,corpus 1,corpus 2,end
0,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,2.1875,"This greenthis is a sontaran, the application line called so recorded and monitored for quality assurance purposes.",agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
1,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,10.3750,May leave your first and your last name.,agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
2,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,11.5625,,client,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
3,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,14.0625,My name is Beth and I was wondering if you could tell me what kind of um.,client,-0.089812,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
4,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,19.0000,What kind of insurance do you? What what you get? What you do? What's your?,client,-0.523072,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096705,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.0625,"Your welcome. I hope I was able to help out with information. You have a great rest of your day, Gustavo, and I'll reach out next week just to follow up, Okay?",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096706,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.1875,The.,client,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096707,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,591.1875,"Alright, sounds good. you have a good one, too.",client,0.956295,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096708,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,594.0625,"Thank you take care. bye, bye.",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827


In [7]:
actual_df = pd.read_excel(r"Aflac Performance Dashboard -  EOM Feb 2023.xlsb")
actual_df

,#,session_id,agent,username,email,team_name,evaluated_by,second_evaluator,workload_name,evaluation_time,...,section_4_answer_6_type,section_4_question_6_comment,section_4_question_7,section_4_question_7_group,section_4_answer_7,section_4_answer_7_score,section_4_answer_7_max_score,section_4_answer_7_type,section_4_question_7_comment,Unnamed: 225
0,40503,FN4O063H7L0U7FAHP16ARNVCAC000EJH_2022-01-02_20-50-17_,Dawn Long,dawn.long@sutherlandglobal.com,dawn.long@sutherlandglobal.com,Sales Team Shawonda,Nikhil Rajan Joseph,NaN,---,3300.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
1,40504,FN4O063H7L0U7FAHP16ARNVCAC000C7N_2022-01-02_19-13-09_,Dawn Long,dawn.long@sutherlandglobal.com,dawn.long@sutherlandglobal.com,Sales Team Shawonda,Nikhil Rajan Joseph,NaN,---,1619.0,...,default,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
2,40505,FN4O063H7L0U7FAHP16ARNVCAC000DNT_2022-01-02_20-12-57_,Richard Gallipeau,richard.gallipeau@sutherlandglobal.com,richard.gallipeau@sutherlandglobal.com,Sales Team Simpkins,Deepa Viswan,NaN,---,1494.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
3,40506,FN4O063H7L0U7FAHP16ARNVCAC00089S_2022-01-02_16-39-28_,Richard Gallipeau,richard.gallipeau@sutherlandglobal.com,richard.gallipeau@sutherlandglobal.com,Sales Team Simpkins,Deepa Viswan,NaN,---,748.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
4,40507,FN4O063H7L0U7FAHP16ARNVCAC0006QI_2022-01-02_15-20-15_,Richard Gallipeau,richard.gallipeau@sutherlandglobal.com,richard.gallipeau@sutherlandglobal.com,Sales Team Simpkins,Deepa Viswan,NaN,---,2125.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8850,50774,7JCCO8JK0T7UH4H1Q3FILGCK2S00GFRN,Donna Williams,donna.williams@sutherlandglobal.com,donna.williams@sutherlandglobal.com,Sales Team Shawonda,Antony Varghese,---,---,1339.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
8851,50775,7JCCO8JK0T7UH4H1Q3FILGCK2S00GN2S,Erica Magana,erica.magana@sutherlandglobal.com,erica.magana@sutherlandglobal.com,Sales Team Shawonda,Antony Varghese,---,---,1170.0,...,not_applicable,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
8852,50777,609U55O62L5GVFFI61EIGIGKI00021JI,Richard Harris,richard.harris1@sutherlandglobal.com,richard.harris1@sutherlandglobal.com,Sales Team Vasquez,Nikhil Rajan Joseph,---,---,3595.0,...,default,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN
8853,50778,7JCCO8JK0T7UH4H1Q3FILGCK2S00H4N9,Brenda Santiago,brenda.santiago@sutherlandglobal.com,brenda.santiago@sutherlandglobal.com,Sales Team Elliott,Antony Varghese,---,---,1375.0,...,default,NaN,Pre-Qualification Questions (Mandatory on All calls)*****,---,Yes,1.0,1,default,NaN,NaN


In [8]:
actual_df = pd.read_excel(r"Aflac Performance Dashboard -  EOM Feb 2023.xlsb")


In [37]:
data_raw

,session_id,start_date,end_date,call_type,agent_id,agent_name,timestamp,chat_text,entity_type,sentiment_score,...,num_transfers,overlap_time,max_silence,sentiment,num_holds,last_speaker,aperture,corpus 1,corpus 2,end
0,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,2.1875,"This greenthis is a sontaran, the application line called so recorded and monitored for quality assurance purposes.",agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
1,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,10.3750,May leave your first and your last name.,agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
2,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,11.5625,,client,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
3,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,14.0625,My name is Beth and I was wondering if you could tell me what kind of um.,client,-0.089812,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
4,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,19.0000,What kind of insurance do you? What what you get? What you do? What's your?,client,-0.523072,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096705,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.0625,"Your welcome. I hope I was able to help out with information. You have a great rest of your day, Gustavo, and I'll reach out next week just to follow up, Okay?",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096706,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.1875,The.,client,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096707,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,591.1875,"Alright, sounds good. you have a good one, too.",client,0.956295,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096708,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,594.0625,"Thank you take care. bye, bye.",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827


In [41]:
data_vch = pd.read_csv(cwd + "\\actorchat_tblactivity_merged.csv")[['session_id','vchproductsold','IStage1','IStage2','IStage3','IStage4']].drop_duplicates()
#data_raw = pd.merge(data_raw, data_vch, left_on=['session_id'], how='inner', right_on=['session_id'])

In [42]:
t = data_vch.session_id.tolist()

In [43]:
data_raw[~data_raw['session_id'].isin(data_vch['session_id'])].session_id.unique().tolist()

[]

In [10]:
data_raw = pd.merge(data_raw, data_vch, left_on=['session_id'], how='inner', right_on=['session_id'])

In [11]:
data_raw.to_csv('all_data.csv')

In [12]:
len(data_vch.session_id.unique())

5410

In [13]:
len(data_vch)

5448

In [16]:
data_vch = data_vch[(data_vch['IStage1'] == 118.0) & (data_vch['vchproductsold'].isnull())]


In [22]:
data_vch

,session_id,vchproductsold,IStage1,IStage2,IStage3,IStage4
0,043LG7223D0UB234B358EQ2F3O000CF2,NaN,118.0,121.0,141.0,147.0
391,043LG7223D0UB234B358EQ2F3O000DOJ,NaN,118.0,121.0,141.0,147.0
520,043LG7223D0UB234B358EQ2F3O000EEA,NaN,118.0,121.0,141.0,147.0
1422,043LG7223D0UB234B358EQ2F3O000II3,NaN,118.0,121.0,141.0,14.0
1764,043LG7223D0UB234B358EQ2F3O000JGV,NaN,118.0,121.0,141.0,147.0
1966,043LG7223D0UB234B358EQ2F3O000JR5,NaN,118.0,121.0,141.0,147.0
2722,043LG7223D0UB234B358EQ2F3O000MM9,NaN,118.0,121.0,141.0,147.0
2951,043LG7223D0UB234B358EQ2F3O000NLL,NaN,118.0,121.0,141.0,147.0
3318,043LG7223D0UB234B358EQ2F3O000O4G,NaN,118.0,121.0,141.0,147.0
3503,043LG7223D0UB234B358EQ2F3O000P3M,NaN,118.0,121.0,141.0,14.0


In [21]:
unique_session_ids = data_vch['session_id'].unique().tolist()

# Convert the list to a DataFrame or Series
unique_session_ids_df = pd.DataFrame({'session_id': unique_session_ids})

# Save the DataFrame to an Excel file
unique_session_ids_df.to_excel('gggg.xlsx', index=False)

In [16]:
data_raw = pd.merge(data_raw, data_vch, left_on=['session_id'], how='inner', right_on=['session_id'])

In [17]:
data_raw.session_id.unique().tolist()

['043LG7223D0UB234B358EQ2F3O000CF2',
 '043LG7223D0UB234B358EQ2F3O000DOJ',
 '043LG7223D0UB234B358EQ2F3O000EEA',
 '043LG7223D0UB234B358EQ2F3O000II3',
 '043LG7223D0UB234B358EQ2F3O000JGV',
 '043LG7223D0UB234B358EQ2F3O000JR5',
 '043LG7223D0UB234B358EQ2F3O000MM9',
 '043LG7223D0UB234B358EQ2F3O000NLL',
 '043LG7223D0UB234B358EQ2F3O000O4G',
 '043LG7223D0UB234B358EQ2F3O000P3M',
 '043LG7223D0UB234B358EQ2F3O000T2G',
 '043LG7223D0UB234B358EQ2F3O000UV7',
 '043LG7223D0UB234B358EQ2F3O00140I',
 '043LG7223D0UB234B358EQ2F3O0016SB',
 '043LG7223D0UB234B358EQ2F3O001BJD',
 '043LG7223D0UB234B358EQ2F3O001KFR',
 '043LG7223D0UB234B358EQ2F3O001QMI',
 '043LG7223D0UB234B358EQ2F3O001TG4',
 '043LG7223D0UB234B358EQ2F3O001V2N',
 '043LG7223D0UB234B358EQ2F3O0028EN',
 '043LG7223D0UB234B358EQ2F3O0029AV',
 '043LG7223D0UB234B358EQ2F3O002AOC',
 '043LG7223D0UB234B358EQ2F3O002DMJ',
 '043LG7223D0UB234B358EQ2F3O002KT3',
 '043LG7223D0UB234B358EQ2F3O002NDV',
 '043LG7223D0UB234B358EQ2F3O002QNO',
 '043LG7223D0UB234B358EQ2F3O002SAL',
 

In [18]:
#data_raw_1 = pd.merge(data_raw[~data_raw['session_id'].isin(actual_df_2['session_id'])], data_vch, left_on=['session_id'], how='inner', right_on=['session_id'])

In [19]:
len(data_raw.session_id.unique())

2626

In [20]:
len(data_raw["session_id"].unique())

2626

In [21]:
result_df = pd.merge(data_raw[["session_id", "chat_text","entity_type","timestamp"
    ]], actual_df, left_on=['session_id'], how='inner', right_on=['session_id'])

In [22]:
follow_up = [
    "(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|when|need me|contact)(.*)(follow up|Follow up|call back|call you?|call you|called you)","(send)(.*)(email|information)(.*)(follow up|Follow up)",
    "(follow up|Follow up)(.*)(with you)","(give you|schedule|arrange|set up)(.*)(call|callback|call back|call you back|call later)","(callback|call back)(.*)(tomorrow)","(give you)(.*)(call|callback|call back|call you back|call later|conv.)",
    "(call you|called you|called you back)(.*)(right back|back|Friday|first week|November)",
    "(set up|schedule|follow|set aside|scheduled|going to|get up|set)(.*)(appointment|appointments|appointments,|appointment,)",
    "(appointment|appointments|appointments,|appointment,)(.*)(reach back|reach out|call you|reminder|reach back out)"
   
]

In [24]:
agent_df = result_df[result_df['entity_type'] == 'agent']
filtered_agent_df = agent_df.groupby('session_id').tail(30)



In [25]:
#m = set(d['session_id'] for _, d in result_df_sorted.iterrows() if d.get('entity_type') == 'agent' and any(re.search(pattern, d['chat_text']) for pattern in (follow_up)))

In [26]:
m = set(d['session_id'] for _, d in filtered_agent_df.iterrows() if any(re.search(pattern, d['chat_text']) for pattern in (follow_up)))

In [27]:
len(m)

1530

In [ ]:
043LG7223D0UB234B358EQ2F3O00140I
043LG7223D0UB234B358EQ2F3O006VOA

In [28]:
m

{'LKL9VL2RUP3SB6QFMUHC830T7000F026',
 '0JI9SKPQQ971528JUUUSCORNTK00AIS4',
 'KK5MPL8GG911R0E1AC5QUIOP70067KBM',
 'L17LEQKG6H2A77JNOPQIFJ6ST40017V0',
 '043LG7223D0UB234B358EQ2F3O0033QQ',
 'L17LEQKG6H2A77JNOPQIFJ6ST400A4D8',
 'KK5MPL8GG911R0E1AC5QUIOP70032VPQ',
 'KK5MPL8GG911R0E1AC5QUIOP7005AJDE',
 'KK5MPL8GG911R0E1AC5QUIOP7002KME1',
 '043LG7223D0UB234B358EQ2F3O006LJR',
 'KK5MPL8GG911R0E1AC5QUIOP7002LRLK',
 '043LG7223D0UB234B358EQ2F3O01LLIV',
 'LKL9VL2RUP3SB6QFMUHC830T7000OAEC',
 '043LG7223D0UB234B358EQ2F3O00QGPS',
 '043LG7223D0UB234B358EQ2F3O001KFR',
 'KK5MPL8GG911R0E1AC5QUIOP7002E71J',
 'KK5MPL8GG911R0E1AC5QUIOP700344BK',
 '043LG7223D0UB234B358EQ2F3O00HVAU',
 'KK5MPL8GG911R0E1AC5QUIOP7005NF6H',
 '0JI9SKPQQ971528JUUUSCORNTK002MT4',
 'SRC7K28UCT42HA3ALE47AJJ584006009',
 'KK5MPL8GG911R0E1AC5QUIOP7002R8VG',
 '043LG7223D0UB234B358EQ2F3O013OHL',
 'KK5MPL8GG911R0E1AC5QUIOP7004USV6',
 '0JI9SKPQQ971528JUUUSCORNTK00LM5P',
 'LKL9VL2RUP3SB6QFMUHC830T7001GN4C',
 '043LG7223D0UB234B358EQ2F3O00V8TN',
 

In [ ]:
['043LG7223D0UB234B358EQ2F3O000P3M',
 '043LG7223D0UB234B358EQ2F3O000UV7',
 '043LG7223D0UB234B358EQ2F3O001QMI',
 '043LG7223D0UB234B358EQ2F3O002AOC',
 '043LG7223D0UB234B358EQ2F3O002DMJ',

In [29]:
data_raw[~data_raw['session_id'].isin(m)].session_id.unique().tolist()

['043LG7223D0UB234B358EQ2F3O000P3M',
 '043LG7223D0UB234B358EQ2F3O000UV7',
 '043LG7223D0UB234B358EQ2F3O001QMI',
 '043LG7223D0UB234B358EQ2F3O002AOC',
 '043LG7223D0UB234B358EQ2F3O002DMJ',
 '043LG7223D0UB234B358EQ2F3O002SAL',
 '043LG7223D0UB234B358EQ2F3O0042GI',
 '043LG7223D0UB234B358EQ2F3O0043VA',
 '043LG7223D0UB234B358EQ2F3O0047QA',
 '043LG7223D0UB234B358EQ2F3O004VVR',
 '043LG7223D0UB234B358EQ2F3O0059MM',
 '043LG7223D0UB234B358EQ2F3O005CGV',
 '043LG7223D0UB234B358EQ2F3O005D2O',
 '043LG7223D0UB234B358EQ2F3O005JMN',
 '043LG7223D0UB234B358EQ2F3O005MRO',
 '043LG7223D0UB234B358EQ2F3O005SP1',
 '043LG7223D0UB234B358EQ2F3O0069T6',
 '043LG7223D0UB234B358EQ2F3O006CG3',
 '043LG7223D0UB234B358EQ2F3O006IA9',
 '043LG7223D0UB234B358EQ2F3O006RLE',
 '043LG7223D0UB234B358EQ2F3O0075GQ',
 '043LG7223D0UB234B358EQ2F3O00802I',
 '043LG7223D0UB234B358EQ2F3O008868',
 '043LG7223D0UB234B358EQ2F3O008AK9',
 '043LG7223D0UB234B358EQ2F3O008DR7',
 '043LG7223D0UB234B358EQ2F3O008F7E',
 '043LG7223D0UB234B358EQ2F3O008GKK',
 

In [ ]:
043LG7223D0UB234B358EQ2F3O006RLE
043LG7223D0UB234B358EQ2F3O00802I

In [ ]:
043LG7223D0UB234B358EQ2F3O000II3
043LG7223D0UB234B358EQ2F3O000MM9
043LG7223D0UB234B358EQ2F3O001BJD

In [155]:
m = {}

for _, d in result_df_sorted.iterrows():
    if d.get('entity_type') == 'agent':
        for pattern in follow_up:
            if re.search(pattern, d['chat_text']):
                session_id = d['session_id']
                if session_id in m:
                    m[session_id].append(pattern)
                else:
                    m[session_id] = [pattern]
                break

print(m)

{'043LG7223D0UB234B358EQ2F3O000II3': ['(set up|schedule|follow|set aside|scheduled|going to|get up|set)(.*)(appointment|appointments|appointments,|appointment,)'], '043LG7223D0UB234B358EQ2F3O000MM9': ["(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|when|need me|contact)(.*)(follow up|Follow up|call back|call you?|call you|called you)", "(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|when|need me|contact)(.*)(follow up|Follow up|call back|call you?|call you|called you)"], '043LG7223D0UB234B358EQ2F3O00140I': ["(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|when|need me|contact)(.*)(follow up|Follow up|call back|call 

In [101]:
len(m)

465

In [102]:
m

{'043LG7223D0UB234B358EQ2F3O000II3': ['(set up|schedule|follow|set aside|scheduled|going to|get up|set)(.*)(appointment|appointments|appointments,|appointment,)'],
 '043LG7223D0UB234B358EQ2F3O000MM9': ["(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|going|when|need me|contact)(.*)(follow up|Follow up|call back|call you?|call you|called you)",
  "(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|going|when|need me|contact)(.*)(follow up|Follow up|call back|call you?|call you|called you)"],
 '043LG7223D0UB234B358EQ2F3O00140I': ["(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later|good time|I'll|what about|going|when|need me|contact)(.*)(follow up|Fol

In [ ]:
661/932----not

In [41]:
filtered_df = actual_df_1[~actual_df_1['session_id'].isin(m)]
filtered_df.to_csv('dashboard_result.csv', index=False)

In [42]:
len(filtered_df.session_id.unique())

3393

In [92]:
actual_df_1.assign(result=actual_df_1['session_id'].isin(m).astype(int)).to_csv('dashboard_result.csv')


In [93]:
actual_df_1[actual_df_1['session_id'].isin(data_raw["session_id"])].assign(result=actual_df_1['session_id'].isin(m).astype(int)).to_csv('dashboard_result.csv')


In [94]:
matched_session_ids = set()

# Loop through the rows of result_df_sorted
for _, d in result_df_sorted.iterrows():
    if d.get('entity_type') == 'agent' and any(re.search(pattern, d['chat_text']) for pattern in follow_up):
        matched_session_ids.add(d['session_id'])
        print("Matched 'chat_text':", d['chat_text'])

# Print the session IDs that have a match in their 'chat_text'
print("Session IDs with a match:", matched_session_ids)
print(len(matched_session_ids))

Matched 'chat_text': Okay, I'll send this information to you and then I'll follow up with you in two weeks, but if you're able to do it earlier, beleive, you call it eight eight eight.
Matched 'chat_text': Okay, Okay, I'm going to send you an email and then I'll follow up you you say in two weeks.
Matched 'chat_text': I was supposed to give you, but you said you send me an email, so I'm trying to prepare this email, but let me give you a call. as I have said I can go over the policy benefits to you real quick if you want to.
Matched 'chat_text': Okay, I'll send you the email And what do you want me to follow up with you then?
Matched 'chat_text': Okay, all right, I will set up a call back for you for around that time, Um tomorrow?
Matched 'chat_text': Well, do you want to set up a callback?
Matched 'chat_text': Okay, you're quite welcome. Would you like me to give you a follow up call some time just to be sure I find out if you were able to get coverage through a group or if you still 

Matched 'chat_text': Yeah, I mean is there a call back date in time I can set up to follow up with you?
Matched 'chat_text': Okay, Um, I will go ahead and email this over to you, and then once you speak to your employer to see if they are able to offer those if they're not, or if they're just offering certain ones, But you want to still sign up for the other programs. Fifty. give me a call back. My name is Nicky last year and I can get set up for you. I'm going to send you a separate email and one with the breakdown of all these different policies, but the second one will have my name and number is you can give me a call back and then if Y.
Matched 'chat_text': Okay, so while I'm setting up this email for you smiles, I'm also going to go ahead and give you my call back my name and my call back number. Because I'm your fakeagent from this point forward. Do you have a pen and paper handy man?
Matched 'chat_text': Alright, you want me to make arrangements to call you back later this week.

Matched 'chat_text': I give you call back at four Pm. My name is William last name short.
Matched 'chat_text': Okay, let's see so in Alabama, the earliest I will be able to give you a call will be nine o'clock central time, so it'll be a little bit later because I'm not able to dial out earlier than that to you, Okay?
Matched 'chat_text': Ah, were you wanting me to set up a call back day in time to follow up with you?
Matched 'chat_text': Okay, and do you want to set up a call back for a later time to give you time to look at everything?
Matched 'chat_text': Okay, just as long as you know something you know for you to think about you know, and I know you. You know you're kind of searching, which is fine, and what we can do is set up a call back having to think it over and I got in the information as well.
Matched 'chat_text': So perfect, so I will give you a call back on August thirty first at four thirty P M your time.
Matched 'chat_text': I am in the office Thursday, and I would be m

Matched 'chat_text': So, uh that just give you a follow call sometime next Thursday, Just touch bases with you. see if you read me forward or not okay.
Matched 'chat_text': Oh, okay, okay, well, then Yeah, I'll go ahead and get this over to you and then if you're interested in moving ahead, just give me a call back. My number name and number will be in the email, or you can even reply back to the email and just say hey, Nicki. give me a call and I'll call you back and we can go go over any questions that you may have okay.
Matched 'chat_text': All right, so how much time do you need to do that and I will give you a call back another day?
Matched 'chat_text': I had to give you a call back.
Matched 'chat_text': Right so I sent both emails the pricing and the brochures, And Uh, when is a good time to give you a follow up call later today. Um, I can reach back out.
Matched 'chat_text': Oh, then you don't need it at all. That's right. Okay, all right, so I'll give you a call back. I'll give

Matched 'chat_text': Okay, no problem, and would you like for me to call you back tomorrow at this time?
Matched 'chat_text': Sure is there a day and time you want me to give you a call back.
Matched 'chat_text': Yeah, so you email me and tell me when the call you. Also. If you want me to call you back a specific day in time. Just tell me that. now I'll put that on my calendar as well.
Matched 'chat_text': Okay, all right then I can try and give you a call. Let's see.
Matched 'chat_text': Yeah, I could call you back tomorrow. I'm here tomorrow and Saturday, but I'm out at six thirty p M Eastern standard time a year in Alabama. I think you are in Eastern Standard, right?
Matched 'chat_text': Okay, so I will go ahead and email some information over to you. Ms. King. Um, What's your schedule like next week? What would be a good day for me to follow up with you in case you had any questions or concerns or you were ready to get involved.
Matched 'chat_text': Not a problem well again, man. M

...

Matched 'chat_text': I'm in the office eight to four thirty Eastern, but I'll give you a call back on Monday if I don't hear from you before that.
Matched 'chat_text': So you don't have to go to the same process again? I can schedule a call back for you.
Matched 'chat_text': Uhhuh, so, when do you want me to follow up with you then?
Matched 'chat_text': Okay, no problem, no problem, so again Mr. Herne does, Um, I just sent all of this information out to your email, so you should have all of your quote information to refer back to whenever you kinda have some time to skim over that email, and then again I do have a set up where a call next Monday. right now. My time it is three fifteen, so I can reach out to you around three Pm.
Matched 'chat_text': Any time after three thirty, Okay, all right, we put it for three, Okay, so I'll give you a. I'll give you a call. Right you shoot them an enroll today.
Matched 'chat_text': Oh, sure thing I'm going to send you an email that has my name ever

Matched 'chat_text': I hear you I'm not in the office tomorrow. I'll be back in the office. Sunday. Um, I could reach out any time after like Sunday or after that, but if you're ready to get the coverage set up tomorrow, you could definitely call back and another agent would be able to help out. Would you like me to give you a call Sunday?
Matched 'chat_text': Okay, so I'll just give you guys a call back at eight o'clock. I'm also going to send you an email. Miss Mc. Kenzie. Um, Well, a few emails with these these prices that I was telling you about Okay for the different plants.
Matched 'chat_text': Okay, well, I could always give you a call back at a better time, but if you want, I can go ahead and get the accident done for you unless you have any further questions about that one.
Matched 'chat_text': So when you're ready to sign up, you just email me and tell me when to call you, Because like I say calling up here is turmoil, So you will have your hands on all our policies. You just

Matched 'chat_text': A with the accident, in particular, I wouldn't want to delay that too much, just because you never know when something's going to be happening. If we're not taking care of it today. I'd be happy to schedule an appointment to give you a call back. This is going to be a question of when might be good for you.
Matched 'chat_text': Well, I sent those to you. When would you like for me to follow up with you? Gregory next week? Give me some time over the weekend and look at it.
Matched 'chat_text': You are so welcome and if I don't hear from you or from you by now on Tuesday, I'll give you a call and say Hey, Cole My future grand daughter's name. How are you doing?
Matched 'chat_text': All right, do you mind if I follow up with you on Monday?
Matched 'chat_text': Perfect, Okay, I'll give you a call Wednesday then and I just sent you the first female. I'm working on the second one now and he'll get it right over to you.
Matched 'chat_text': Yes, ma'am, so, Uh, Yeah, I dis

Matched 'chat_text': When we make arrangements to call you back in a couple of days or.
Matched 'chat_text': All right again, my name is Eric. Lester. You have a good rest of your evening and I'll give you a call back at nine thirty.
Matched 'chat_text': If you want to call back tonight, I can actually just set up the call about four year. It's easier for me just to call you versus you call in. Cause it's a call center. so we don't have direct lines. someone to transfer you to. I'll put you on a call back now. Um what time would you like that call back?
Matched 'chat_text': Okay, but if you're able to do it earlier, you can certainly give you a call.
Matched 'chat_text': I'll probably give you a call back. Say the same time, right?
Matched 'chat_text': Ah, do you have any other questions or do you want me to set up a call back with you for later on today?
Matched 'chat_text': Okay, Yeah, if you'd like, I'd be happy to give you a call back around seven on Wednesday. No, I do just want t

Matched 'chat_text': Okay, I am in a call center. so I. I. I'm I'm a schedule for eleven o'clock tomorrow. Uh, actually eleven ten tomorrow at eleven o'clock So eleventhhour. If I don't call you right at eleven ten. as soon as I get off the phone, I'll give you a call back. I work Monday through Friday from nine to five thirty Eastern standard time. My number here is eight, eight eight.
Matched 'chat_text': Okay, I can definitely give you a call tomorrow. It's a good time.
Matched 'chat_text': So let me schedule a call back with you. Okay? Um what time did you want me to call you back? So what would you have time?
Matched 'chat_text': Uh, what's a good call back dating time I could set up to follow up with you?
Matched 'chat_text': No, actually, that's for all agents. If I don't pick up the phone, Because that is used by all agents. Just give my name to the person who picks up and you will be transferred to me. If I am on the other line. They'll send me a quick message to let me know t

In [96]:
m = set(d['session_id'] for _, d in result_df.iterrows() if d.get('entity_type') == 'agent' and any(re.search(pattern, d['chat_text']) for pattern in (follow_up)))

In [201]:
#result_df[(result_df['IStage1'] == 118) & (~result_df['session_id'].isin(m))].to_excel(cwd + "//result_1234.xlsx")

In [ ]:
result_df[(result_df['IStage1'] == 118) & (~result_df['session_id'].isin(m))].session_id.unique().tolist()

In [ ]:
len(result_df[~result_df['session_id'].isin(not_met_list)].session_id.unique())

In [ ]:
   Pred      1       0
Actual   1  1141    286   
         0   551    639     

In [ ]:
2617---1780

In [ ]:
 '7JCCO8JK0T7UH4H1Q3FILGCK2S003HDF',   1  Is there a number that I could call you back? Would it be?
 'LKL9VL2RUP3SB6QFMUHC830T70018SED',   1
 '7JCCO8JK0T7UH4H1Q3FILGCK2S0087JF',   1  Right, I understood what I can. If you want me to, I could give you a call in about a week to see if you found anything with that work.
 'L17LEQKG6H2A77JNOPQIFJ6ST400B9P0',   1 Okay, I guess now I can give you a call on Tuesday Wednesday if you like.
 'KK5MPL8GG911R0E1AC5QUIOP70028IAJ',   1 Tomorrow or Sunday, I mean you're not going to be available on Sunday, so you get a call back on tomorrow afternoon.
 '0JI9SKPQQ971528JUUUSCORNTK00QA07',   1

In [ ]:
{'043LG7223D0UB234B358EQ2F3O00E3RS',   11

 'LKL9VL2RUP3SB6QFMUHC830T70008T2G',   11

 '043LG7223D0UB234B358EQ2F3O01270N',   11
 
 '043LG7223D0UB234B358EQ2F3O00CBQ2',   11
 '7JCCO8JK0T7UH4H1Q3FILGCK2S00FI3F',   11
 'KK5MPL8GG911R0E1AC5QUIOP7002EDGR',   11

 '043LG7223D0UB234B358EQ2F3O00F944',   11
 'KK5MPL8GG911R0E1AC5QUIOP7002J982',   11
 '7JCCO8JK0T7UH4H1Q3FILGCK2S001MCE',   11

 'KK5MPL8GG911R0E1AC5QUIOP7005LPKB',   11
 'KK5MPL8GG911R0E1AC5QUIOP700601AF',   11

 'LKL9VL2RUP3SB6QFMUHC830T7001D2DV',   11
 '043LG7223D0UB234B358EQ2F3O000II3',
 '043LG7223D0UB234B358EQ2F3O000MM9',
 '043LG7223D0UB234B358EQ2F3O00140I',
 '043LG7223D0UB234B358EQ2F3O001BJD',
 '043LG7223D0UB234B358EQ2F3O001V2N',
 '043LG7223D0UB234B358EQ2F3O0042GI',
 '043LG7223D0UB234B358EQ2F3O0047QA',
 '043LG7223D0UB234B358EQ2F3O005CGV',
 '043LG7223D0UB234B358EQ2F3O005NB5',
 '043LG7223D0UB234B358EQ2F3O005VFM',
 '043LG7223D0UB234B358EQ2F3O006LJR',
 '043LG7223D0UB234B358EQ2F3O006NTF',
 '043LG7223D0UB234B358EQ2F3O006VOA',
 '043LG7223D0UB234B358EQ2F3O00888D',
 '043LG7223D0UB234B358EQ2F3O008AK9',
 '043LG7223D0UB234B358EQ2F3O009NN3',
 '043LG7223D0UB234B358EQ2F3O00BUUV',
 '043LG7223D0UB234B358EQ2F3O00C6B0',
 '043LG7223D0UB234B358EQ2F3O00CAHS',
 '043LG7223D0UB234B358EQ2F3O00CBQ2',
 '043LG7223D0UB234B358EQ2F3O00COOJ',
 '043LG7223D0UB234B358EQ2F3O00CS33',
 '043LG7223D0UB234B358EQ2F3O00D9UN',
 '043LG7223D0UB234B358EQ2F3O00DMEN',
 '043LG7223D0UB234B358EQ2F3O00E62P',
 '043LG7223D0UB234B358EQ2F3O00E7J3',
 '043LG7223D0UB234B358EQ2F3O00F1M2',
 '043LG7223D0UB234B358EQ2F3O00F779',
 '043LG7223D0UB234B358EQ2F3O00F81V',
 '043LG7223D0UB234B358EQ2F3O00F944',
 '043LG7223D0UB234B358EQ2F3O00FCFJ',
 '043LG7223D0UB234B358EQ2F3O00FDQB',
 '043LG7223D0UB234B358EQ2F3O00FHNG',
 '043LG7223D0UB234B358EQ2F3O00FJPT',
 '043LG7223D0UB234B358EQ2F3O00G3VC',
 '043LG7223D0UB234B358EQ2F3O00GEJT',
 '043LG7223D0UB234B358EQ2F3O00GFSD',
 '043LG7223D0UB234B358EQ2F3O00GOQQ',
 '043LG7223D0UB234B358EQ2F3O00HKCR',
 '043LG7223D0UB234B358EQ2F3O00HM2U',
 '043LG7223D0UB234B358EQ2F3O00HR9H',
 '043LG7223D0UB234B358EQ2F3O00I692',
 '043LG7223D0UB234B358EQ2F3O00I8PS',
 '043LG7223D0UB234B358EQ2F3O00I8S0',
 '043LG7223D0UB234B358EQ2F3O00I9M8',
 '043LG7223D0UB234B358EQ2F3O00JO6I',
 '043LG7223D0UB234B358EQ2F3O00JQ6N',
 '043LG7223D0UB234B358EQ2F3O00K2VE',
 '043LG7223D0UB234B358EQ2F3O00KL41',
 '043LG7223D0UB234B358EQ2F3O00KL51',
 '043LG7223D0UB234B358EQ2F3O00KOPJ',
 '043LG7223D0UB234B358EQ2F3O00L3CA',
 '043LG7223D0UB234B358EQ2F3O00LGOK',
 '043LG7223D0UB234B358EQ2F3O00NBTM',
 '043LG7223D0UB234B358EQ2F3O00QBRL',
 '043LG7223D0UB234B358EQ2F3O00QEVB',
 '043LG7223D0UB234B358EQ2F3O00QJ2R',
 '043LG7223D0UB234B358EQ2F3O00QRAG',
 '043LG7223D0UB234B358EQ2F3O00QTUO',
 '043LG7223D0UB234B358EQ2F3O00R6EV',
 '043LG7223D0UB234B358EQ2F3O00RB4C',
 '043LG7223D0UB234B358EQ2F3O00RM0T',
 '043LG7223D0UB234B358EQ2F3O00RNES',
 '043LG7223D0UB234B358EQ2F3O00TED5',
 '043LG7223D0UB234B358EQ2F3O00THFJ',
 '043LG7223D0UB234B358EQ2F3O00U0UE',
 '043LG7223D0UB234B358EQ2F3O00U5EG',
 '043LG7223D0UB234B358EQ2F3O00V8TN',
 '043LG7223D0UB234B358EQ2F3O00VJA5',
 '043LG7223D0UB234B358EQ2F3O01106Q',
 '043LG7223D0UB234B358EQ2F3O0125IE',
 '043LG7223D0UB234B358EQ2F3O01270N',
 '043LG7223D0UB234B358EQ2F3O012CEV',
 '043LG7223D0UB234B358EQ2F3O012DDA',
 '043LG7223D0UB234B358EQ2F3O012H7L',
 '043LG7223D0UB234B358EQ2F3O012MOJ',
 '043LG7223D0UB234B358EQ2F3O012MTC',
 '043LG7223D0UB234B358EQ2F3O012NMV',
 '043LG7223D0UB234B358EQ2F3O012SUI',
 '043LG7223D0UB234B358EQ2F3O01446D',
 '043LG7223D0UB234B358EQ2F3O0146OR',
 '043LG7223D0UB234B358EQ2F3O014BME',
 '043LG7223D0UB234B358EQ2F3O014BV1',
 '043LG7223D0UB234B358EQ2F3O014EFJ',
 '043LG7223D0UB234B358EQ2F3O015GPV',
 '043LG7223D0UB234B358EQ2F3O016J3D',
 

In [ ]:
1. Effort from agent to gain commitment for Follow-up call (If un-successful in 1st touch) –
 I can call you at your convenient time tomorrow to continue, tell me when your spouse will be available so we can get his/her views…  
When can I call you back?
are you going to be available for the rest of the day? So are you just available today at a certain time?
I can just call you back at a better time. Is there a better time for me to call you back mam?
If that works for you, I'll call you back at five.
is there a good time tomorrow to reach out to the two of you regarding these two plants?
What what would be the time for me to follow up with you?


In [2]:
follow_up = [
    "(call back|callback|email|give you|good time|it okay|day and time|schedule|you mind|like me|good day|setting up|reach out|call you back|call later)(.*)(follow up|Follow up)","(send)(.*)(email|information)(.*)(follow up|Follow up)",
    "(follow up|Follow up)(.*)(with you)","(give you|schedule|arrange|set up|email you|want|better time.|better time|work out a time)(.*)(call|callback|call back|call you back|call later|call you back?|call you.)","(callback|call back|time works|catch up)(.*)(tomorrow|tomorrow?|next week?)","(give you|good time)(.*)(call|call you?|callback|call back|call you back|call later|conv.)",
    "(call you|called you)(.*)(right back|back|in a week|my schedule|first week|back?|Thursday)","(convenient time tomorrow|good time tomorrow)","()"
   
]

In [ ]:
043LG7223D0UB234B358EQ2F3O000ELS',         1
 '043LG7223D0UB234B358EQ2F3O000II3',       0 Is it best to email you or call you back?
 '043LG7223D0UB234B358EQ2F3O000P3M',       1
 '043LG7223D0UB234B358EQ2F3O000PRG',       1
 '043LG7223D0UB234B358EQ2F3O000QV0',       1
 '043LG7223D0UB234B358EQ2F3O000S0T',       1
 '043LG7223D0UB234B358EQ2F3O000UV7',       1
 '043LG7223D0UB234B358EQ2F3O0013JL',       1
 '043LG7223D0UB234B358EQ2F3O00140I',       1
 '043LG7223D0UB234B358EQ2F3O00145S',       1
 '043LG7223D0UB234B358EQ2F3O001QMI',       1
 '043LG7223D0UB234B358EQ2F3O001T1J',       1
 '043LG7223D0UB234B358EQ2F3O001V2N',       1
 '043LG7223D0UB234B358EQ2F3O00245U',       1
 '043LG7223D0UB234B358EQ2F3O0029AV',       1
 '043LG7223D0UB234B358EQ2F3O0029N3',       1
 '043LG7223D0UB234B358EQ2F3O002AOC',       1
 '043LG7223D0UB234B358EQ2F3O002B1T',       0  And then gender is female. Yeah, and I heard the phone going a little staticy. If we get disconnected, I'll call you right back because we don't have personal expenses here because we don't work on commission now.
 '043LG7223D0UB234B358EQ2F3O002DMJ',       1
 '043LG7223D0UB234B358EQ2F3O002REU',       1
 '043LG7223D0UB234B358EQ2F3O002SAL',       1
 '043LG7223D0UB234B358EQ2F3O0037HA',       1
 '043LG7223D0UB234B358EQ2F3O00397V',       1
 '043LG7223D0UB234B358EQ2F3O003PLU',       1
 '043LG7223D0UB234B358EQ2F3O0043VA',       1
 '043LG7223D0UB234B358EQ2F3O00470E',       1
 '043LG7223D0UB234B358EQ2F3O0047QA',       1
 '043LG7223D0UB234B358EQ2F3O004VVR',       1
 '043LG7223D0UB234B358EQ2F3O0059MM',       1
 '043LG7223D0UB234B358EQ2F3O005AFQ',       0   Well, what is a good time for me to call you? is probably easier if I just do that on my end was a good time for me to contact you.
 '043LG7223D0UB234B358EQ2F3O005D2O',       1
 '043LG7223D0UB234B358EQ2F3O005JMN',       1
 '043LG7223D0UB234B358EQ2F3O005MRO',       1
 '043LG7223D0UB234B358EQ2F3O005NB5',       1
 '043LG7223D0UB234B358EQ2F3O005OSK',       1 I'm gonna send you this. I'm going to send you the quote and then if you want me to call you back in a week or so.
 '043LG7223D0UB234B358EQ2F3O005SP1',       1
 '043LG7223D0UB234B358EQ2F3O0065EJ',       1
 '043LG7223D0UB234B358EQ2F3O0069T6',       1
 '043LG7223D0UB234B358EQ2F3O006CG3',       1
 '043LG7223D0UB234B358EQ2F3O006IA9',       1
 '043LG7223D0UB234B358EQ2F3O006LJR',       1
 '043LG7223D0UB234B358EQ2F3O006RLE',       1
 '043LG7223D0UB234B358EQ2F3O006RMJ',       1
 '043LG7223D0UB234B358EQ2F3O006SAO',       1
 '043LG7223D0UB234B358EQ2F3O006VO9',       1  Alright, Well, you drive safe, my dear and I'll call you back in an hour. Okay, maybe a little after if I'm on the phone, Don't panic. I didn't forget you and put you on my schedule.
 '043LG7223D0UB234B358EQ2F3O00744T',       1
 '043LG7223D0UB234B358EQ2F3O0075GQ',       1
 '043LG7223D0UB234B358EQ2F3O0075R1',       1 Oh, congratulations, so I'll tell you what about if I called you back like the first week of November.,So I sent you some more quotes and I'm going to call you back on November four.
 '043LG7223D0UB234B358EQ2F3O0079A8',       1
 '043LG7223D0UB234B358EQ2F3O00802I',       1-----------
 '043LG7223D0UB234B358EQ2F3O008868',       1
 '043LG7223D0UB234B358EQ2F3O00888D',       1 And tell me when to call you.
 '043LG7223D0UB234B358EQ2F3O008AK9',       1 And if it's easier for you, I can follow back up with you, as well anything else you need me to do.
 '043LG7223D0UB234B358EQ2F3O008DR7',
 '043LG7223D0UB234B358EQ2F3O008ERL',
 '043LG7223D0UB234B358EQ2F3O008F7E',
 '043LG7223D0UB234B358EQ2F3O008GKK',
 '043LG7223D0UB234B358EQ2F3O008I60',
 '043LG7223D0UB234B358EQ2F3O008UC2',
 '043LG7223D0UB234B358EQ2F3O0098QC',
 '043LG7223D0UB234B358EQ2F3O0099L2',
 '043LG7223D0UB234B358EQ2F3O009I49',
 '043LG7223D0UB234B358EQ2F3O009IQ1',
 '043LG7223D0UB234B358EQ2F3O009L55',
 '043LG7223D0UB234B358EQ2F3O009MSM',
 '043LG7223D0UB234B358EQ2F3O00B0G6',
 '043LG7223D0UB234B358EQ2F3O00B4S0',
 '043LG7223D0UB234B358EQ2F3O00B5O5',
 '043LG7223D0UB234B358EQ2F3O00B65T',
 '043LG7223D0UB234B358EQ2F3O00B6AE',
 '043LG7223D0UB234B358EQ2F3O00B6HR',
 '043LG7223D0UB234B358EQ2F3O00B9DM',
 '043LG7223D0UB234B358EQ2F3O00BFUS',
 '043LG7223D0UB234B358EQ2F3O00BH0J',
 '043LG7223D0UB234B358EQ2F3O00BH19',
 '043LG7223D0UB234B358EQ2F3O00BHB8',
 '043LG7223D0UB234B358EQ2F3O00BIIA',
 '043LG7223D0UB234B358EQ2F3O00BV5R',
 '043LG7223D0UB234B358EQ2F3O00BVPC',
 '043LG7223D0UB234B358EQ2F3O00C3RL',
 '043LG7223D0UB234B358EQ2F3O00C7V8',
 '043LG7223D0UB234B358EQ2F3O00CBFL',
 '043LG7223D0UB234B358EQ2F3O00CC04',
 '043LG7223D0UB234B358EQ2F3O00CG86',
 '043LG7223D0UB234B358EQ2F3O00CHDQ',
 '043LG7223D0UB234B358EQ2F3O00CI3V',
 '043LG7223D0UB234B358EQ2F3O00CJPE',
 '043LG7223D0UB234B358EQ2F3O00CKGK',
 '043LG7223D0UB234B358EQ2F3O00CMD0',
 '043LG7223D0UB234B358EQ2F3O00CMVE',
 '043LG7223D0UB234B358EQ2F3O00CP55',
 '043LG7223D0UB234B358EQ2F3O00CR80',
 '043LG7223D0UB234B358EQ2F3O00DB92',
 '043LG7223D0UB234B358EQ2F3O00DC6M',
 '043LG7223D0UB234B358EQ2F3O00DEH8',
 '043LG7223D0UB234B358EQ2F3O00DFO0',
 '043LG7223D0UB234B358EQ2F3O00DKHO',
 '043LG7223D0UB234B358EQ2F3O00DNM3',
 '043LG7223D0UB234B358EQ2F3O00DVSV',
 '043LG7223D0UB234B358EQ2F3O00E3AD',
 '043LG7223D0UB234B358EQ2F3O00E7G0',
 '043LG7223D0UB234B358EQ2F3O00E7J3',
 '043LG7223D0UB234B358EQ2F3O00E7K5',
 '043LG7223D0UB234B358EQ2F3O00E89C',
 '043LG7223D0UB234B358EQ2F3O00E8KR',
 '043LG7223D0UB234B358EQ2F3O00E8UI',
 '043LG7223D0UB234B358EQ2F3O00ECHL',
 '043LG7223D0UB234B358EQ2F3O00EK07',
 '043LG7223D0UB234B358EQ2F3O00ELNO',
 '043LG7223D0UB234B358EQ2F3O00ENMU',


In [30]:
A1 = ['L17LEQKG6H2A77JNOPQIFJ6ST40098D1', '043LG7223D0UB234B358EQ2F3O00BH0J', 'KK5MPL8GG911R0E1AC5QUIOP7003D4TP', '043LG7223D0UB234B358EQ2F3O00EB9I', 'L17LEQKG6H2A77JNOPQIFJ6ST400B9P0', 'L17LEQKG6H2A77JNOPQIFJ6ST40025P8', 'KK5MPL8GG911R0E1AC5QUIOP7002BA5D', '7JCCO8JK0T7UH4H1Q3FILGCK2S002STA', 'KK5MPL8GG911R0E1AC5QUIOP7004JH8Q', 'SRC7K28UCT42HA3ALE47AJJ58400284N', 'KK5MPL8GG911R0E1AC5QUIOP70028KQ5', '043LG7223D0UB234B358EQ2F3O000JGV', 'LKL9VL2RUP3SB6QFMUHC830T7000E5K9', 'KK5MPL8GG911R0E1AC5QUIOP7002M4O1', 'KK5MPL8GG911R0E1AC5QUIOP7005UF7E', '043LG7223D0UB234B358EQ2F3O00JB2I', 'KK5MPL8GG911R0E1AC5QUIOP7005MUCB', 'SRC7K28UCT42HA3ALE47AJJ58400788K', '2OOPPKETML4Q56CSFN0AVV2VP800EDV7', 'KK5MPL8GG911R0E1AC5QUIOP7003ICSQ', '043LG7223D0UB234B358EQ2F3O01L8P9', 'KK5MPL8GG911R0E1AC5QUIOP7002N60J', '0JI9SKPQQ971528JUUUSCORNTK00VCA9', '043LG7223D0UB234B358EQ2F3O00VBTM', 'LKL9VL2RUP3SB6QFMUHC830T7001BBQK', '0JI9SKPQQ971528JUUUSCORNTK00775E', 'KK5MPL8GG911R0E1AC5QUIOP7003TPNA', 'L17LEQKG6H2A77JNOPQIFJ6ST40086UD', 'LKL9VL2RUP3SB6QFMUHC830T70012T9N', '043LG7223D0UB234B358EQ2F3O00HR9H', 'KK5MPL8GG911R0E1AC5QUIOP7002GGO8', 'KK5MPL8GG911R0E1AC5QUIOP70064711', 'LKL9VL2RUP3SB6QFMUHC830T7000HT0R', 'KK5MPL8GG911R0E1AC5QUIOP70032LCQ', '043LG7223D0UB234B358EQ2F3O00T8HE', 'KK5MPL8GG911R0E1AC5QUIOP7004TQ9T', '0593P4GK7957T82NR3I4UJBIFK0000TE', 'KK5MPL8GG911R0E1AC5QUIOP70027ITU', '0JI9SKPQQ971528JUUUSCORNTK002JER', 'CNBK6R9AN131LF2UDUJP6NB34G00N7L7', 'LKL9VL2RUP3SB6QFMUHC830T7001I7DD', '0JI9SKPQQ971528JUUUSCORNTK012T9M', 'L17LEQKG6H2A77JNOPQIFJ6ST400A8EC', 'LKL9VL2RUP3SB6QFMUHC830T7001HAOK', 'KK5MPL8GG911R0E1AC5QUIOP7005S0IP', '0JI9SKPQQ971528JUUUSCORNTK009US8', '043LG7223D0UB234B358EQ2F3O014UBQ', 'KK5MPL8GG911R0E1AC5QUIOP70059CVH', '043LG7223D0UB234B358EQ2F3O00VKLC', 'L17LEQKG6H2A77JNOPQIFJ6ST400IO1B', '043LG7223D0UB234B358EQ2F3O000II3', '043LG7223D0UB234B358EQ2F3O000MM9', '043LG7223D0UB234B358EQ2F3O00140I', '043LG7223D0UB234B358EQ2F3O001BJD', '043LG7223D0UB234B358EQ2F3O001V2N', '043LG7223D0UB234B358EQ2F3O005NB5', '043LG7223D0UB234B358EQ2F3O005VFM', '043LG7223D0UB234B358EQ2F3O006LJR', '043LG7223D0UB234B358EQ2F3O006NTF', '043LG7223D0UB234B358EQ2F3O006VOA', '043LG7223D0UB234B358EQ2F3O00888D', '043LG7223D0UB234B358EQ2F3O009NN3', '043LG7223D0UB234B358EQ2F3O00BUUV', '043LG7223D0UB234B358EQ2F3O00C6B0', '043LG7223D0UB234B358EQ2F3O00CAHS', '043LG7223D0UB234B358EQ2F3O00CBQ2', '043LG7223D0UB234B358EQ2F3O00COOJ', '043LG7223D0UB234B358EQ2F3O00CS33', '043LG7223D0UB234B358EQ2F3O00D9UN', '043LG7223D0UB234B358EQ2F3O00E62P', '043LG7223D0UB234B358EQ2F3O00E7J3']





In [31]:
A1

['L17LEQKG6H2A77JNOPQIFJ6ST40098D1',
 '043LG7223D0UB234B358EQ2F3O00BH0J',
 'KK5MPL8GG911R0E1AC5QUIOP7003D4TP',
 '043LG7223D0UB234B358EQ2F3O00EB9I',
 'L17LEQKG6H2A77JNOPQIFJ6ST400B9P0',
 'L17LEQKG6H2A77JNOPQIFJ6ST40025P8',
 'KK5MPL8GG911R0E1AC5QUIOP7002BA5D',
 '7JCCO8JK0T7UH4H1Q3FILGCK2S002STA',
 'KK5MPL8GG911R0E1AC5QUIOP7004JH8Q',
 'SRC7K28UCT42HA3ALE47AJJ58400284N',
 'KK5MPL8GG911R0E1AC5QUIOP70028KQ5',
 '043LG7223D0UB234B358EQ2F3O000JGV',
 'LKL9VL2RUP3SB6QFMUHC830T7000E5K9',
 'KK5MPL8GG911R0E1AC5QUIOP7002M4O1',
 'KK5MPL8GG911R0E1AC5QUIOP7005UF7E',
 '043LG7223D0UB234B358EQ2F3O00JB2I',
 'KK5MPL8GG911R0E1AC5QUIOP7005MUCB',
 'SRC7K28UCT42HA3ALE47AJJ58400788K',
 '2OOPPKETML4Q56CSFN0AVV2VP800EDV7',
 'KK5MPL8GG911R0E1AC5QUIOP7003ICSQ',
 '043LG7223D0UB234B358EQ2F3O01L8P9',
 'KK5MPL8GG911R0E1AC5QUIOP7002N60J',
 '0JI9SKPQQ971528JUUUSCORNTK00VCA9',
 '043LG7223D0UB234B358EQ2F3O00VBTM',
 'LKL9VL2RUP3SB6QFMUHC830T7001BBQK',
 '0JI9SKPQQ971528JUUUSCORNTK00775E',
 'KK5MPL8GG911R0E1AC5QUIOP7003TPNA',
 

In [32]:
A2 = ['043LG7223D0UB234B358EQ2F3O00F1M2', '043LG7223D0UB234B358EQ2F3O00F779', '043LG7223D0UB234B358EQ2F3O00F81V', '043LG7223D0UB234B358EQ2F3O00F944', '043LG7223D0UB234B358EQ2F3O00FCFJ', '043LG7223D0UB234B358EQ2F3O00FDQB', '043LG7223D0UB234B358EQ2F3O00FHNG', '043LG7223D0UB234B358EQ2F3O00FJPT', '043LG7223D0UB234B358EQ2F3O00G3VC', '043LG7223D0UB234B358EQ2F3O00GEJT', '043LG7223D0UB234B358EQ2F3O00GFSD', '043LG7223D0UB234B358EQ2F3O00GOQQ', '043LG7223D0UB234B358EQ2F3O00HKCR', '043LG7223D0UB234B358EQ2F3O00HM2U', '043LG7223D0UB234B358EQ2F3O00HR9H', '043LG7223D0UB234B358EQ2F3O00I692', '043LG7223D0UB234B358EQ2F3O00I8PS', '043LG7223D0UB234B358EQ2F3O00I8S0', '043LG7223D0UB234B358EQ2F3O00JO6I', '043LG7223D0UB234B358EQ2F3O00JQ6N', '043LG7223D0UB234B358EQ2F3O00K2VE', '043LG7223D0UB234B358EQ2F3O00KL51', '043LG7223D0UB234B358EQ2F3O00KOPJ', '043LG7223D0UB234B358EQ2F3O00L3CA', '043LG7223D0UB234B358EQ2F3O00LGOK', '043LG7223D0UB234B358EQ2F3O00NBTM', '043LG7223D0UB234B358EQ2F3O00QBRL', '043LG7223D0UB234B358EQ2F3O00QEVB', '043LG7223D0UB234B358EQ2F3O00QJ2R', '043LG7223D0UB234B358EQ2F3O00QTUO', '043LG7223D0UB234B358EQ2F3O00RB4C', '043LG7223D0UB234B358EQ2F3O00RM0T', '043LG7223D0UB234B358EQ2F3O00RNES', '043LG7223D0UB234B358EQ2F3O00TED5', '043LG7223D0UB234B358EQ2F3O00THFJ', '043LG7223D0UB234B358EQ2F3O00V8TN', '043LG7223D0UB234B358EQ2F3O01106Q', '043LG7223D0UB234B358EQ2F3O0125IE', '043LG7223D0UB234B358EQ2F3O012CEV', '043LG7223D0UB234B358EQ2F3O012DDA', '043LG7223D0UB234B358EQ2F3O012H7L', '043LG7223D0UB234B358EQ2F3O012MOJ', '043LG7223D0UB234B358EQ2F3O012MTC', '043LG7223D0UB234B358EQ2F3O012NMV', '043LG7223D0UB234B358EQ2F3O01446D', '043LG7223D0UB234B358EQ2F3O0146OR', '043LG7223D0UB234B358EQ2F3O014BME', '043LG7223D0UB234B358EQ2F3O016J3D', '043LG7223D0UB234B358EQ2F3O01B4SS', '043LG7223D0UB234B358EQ2F3O01CKIE', '043LG7223D0UB234B358EQ2F3O01CUGR', '043LG7223D0UB234B358EQ2F3O01DI9Q', '043LG7223D0UB234B358EQ2F3O01DJF0', '043LG7223D0UB234B358EQ2F3O01E05N', '043LG7223D0UB234B358EQ2F3O000P3M', '043LG7223D0UB234B358EQ2F3O000UV7', '043LG7223D0UB234B358EQ2F3O001QMI', '043LG7223D0UB234B358EQ2F3O002AOC', '043LG7223D0UB234B358EQ2F3O0042GI', '043LG7223D0UB234B358EQ2F3O0043VA', '043LG7223D0UB234B358EQ2F3O0047QA']





In [33]:
A2

['043LG7223D0UB234B358EQ2F3O00F1M2',
 '043LG7223D0UB234B358EQ2F3O00F779',
 '043LG7223D0UB234B358EQ2F3O00F81V',
 '043LG7223D0UB234B358EQ2F3O00F944',
 '043LG7223D0UB234B358EQ2F3O00FCFJ',
 '043LG7223D0UB234B358EQ2F3O00FDQB',
 '043LG7223D0UB234B358EQ2F3O00FHNG',
 '043LG7223D0UB234B358EQ2F3O00FJPT',
 '043LG7223D0UB234B358EQ2F3O00G3VC',
 '043LG7223D0UB234B358EQ2F3O00GEJT',
 '043LG7223D0UB234B358EQ2F3O00GFSD',
 '043LG7223D0UB234B358EQ2F3O00GOQQ',
 '043LG7223D0UB234B358EQ2F3O00HKCR',
 '043LG7223D0UB234B358EQ2F3O00HM2U',
 '043LG7223D0UB234B358EQ2F3O00HR9H',
 '043LG7223D0UB234B358EQ2F3O00I692',
 '043LG7223D0UB234B358EQ2F3O00I8PS',
 '043LG7223D0UB234B358EQ2F3O00I8S0',
 '043LG7223D0UB234B358EQ2F3O00JO6I',
 '043LG7223D0UB234B358EQ2F3O00JQ6N',
 '043LG7223D0UB234B358EQ2F3O00K2VE',
 '043LG7223D0UB234B358EQ2F3O00KL51',
 '043LG7223D0UB234B358EQ2F3O00KOPJ',
 '043LG7223D0UB234B358EQ2F3O00L3CA',
 '043LG7223D0UB234B358EQ2F3O00LGOK',
 '043LG7223D0UB234B358EQ2F3O00NBTM',
 '043LG7223D0UB234B358EQ2F3O00QBRL',
 

In [34]:
A3 = [
'043LG7223D0UB234B358EQ2F3O004VVR',
'043LG7223D0UB234B358EQ2F3O0059MM',
'043LG7223D0UB234B358EQ2F3O005CGV',
'043LG7223D0UB234B358EQ2F3O005D2O',
'043LG7223D0UB234B358EQ2F3O005MRO',
'043LG7223D0UB234B358EQ2F3O005SP1',
'043LG7223D0UB234B358EQ2F3O0069T6',
'043LG7223D0UB234B358EQ2F3O006CG3',
'043LG7223D0UB234B358EQ2F3O006IA9',
'043LG7223D0UB234B358EQ2F3O006RLE',
'043LG7223D0UB234B358EQ2F3O0075GQ',
'043LG7223D0UB234B358EQ2F3O00802I',
'043LG7223D0UB234B358EQ2F3O008868',
'043LG7223D0UB234B358EQ2F3O008AK9',
'043LG7223D0UB234B358EQ2F3O008DR7',
'043LG7223D0UB234B358EQ2F3O008GKK',
'043LG7223D0UB234B358EQ2F3O0099L2',
'043LG7223D0UB234B358EQ2F3O00B5O5',
'043LG7223D0UB234B358EQ2F3O00B65T',
'043LG7223D0UB234B358EQ2F3O00B6AE',
'043LG7223D0UB234B358EQ2F3O00BH19',
'043LG7223D0UB234B358EQ2F3O00BHB8',
'043LG7223D0UB234B358EQ2F3O00BIIA',
'043LG7223D0UB234B358EQ2F3O00BV5R',
'043LG7223D0UB234B358EQ2F3O00BVPC',
'043LG7223D0UB234B358EQ2F3O00CC04',
'043LG7223D0UB234B358EQ2F3O00CMD0',
'043LG7223D0UB234B358EQ2F3O00CMVE',
'043LG7223D0UB234B358EQ2F3O00CP55',
'043LG7223D0UB234B358EQ2F3O00D7CD',
'043LG7223D0UB234B358EQ2F3O00DB92',
'043LG7223D0UB234B358EQ2F3O00DC6M',
'043LG7223D0UB234B358EQ2F3O00DFO0',
'043LG7223D0UB234B358EQ2F3O00DMEN',
'043LG7223D0UB234B358EQ2F3O00E3AD',
'043LG7223D0UB234B358EQ2F3O00E7G0',
'043LG7223D0UB234B358EQ2F3O00E89C',
'043LG7223D0UB234B358EQ2F3O00E8KR',
'043LG7223D0UB234B358EQ2F3O00E8UI',
'043LG7223D0UB234B358EQ2F3O00EK07',
'043LG7223D0UB234B358EQ2F3O00ET8G',
'043LG7223D0UB234B358EQ2F3O00F3IJ',
'043LG7223D0UB234B358EQ2F3O00F6QG',
'043LG7223D0UB234B358EQ2F3O00FICV',
'043LG7223D0UB234B358EQ2F3O00GGF2',
'043LG7223D0UB234B358EQ2F3O00GK66',
'043LG7223D0UB234B358EQ2F3O00H14L',
'043LG7223D0UB234B358EQ2F3O00H6IT',
'043LG7223D0UB234B358EQ2F3O00HKO0',
'043LG7223D0UB234B358EQ2F3O00I9D6',
'043LG7223D0UB234B358EQ2F3O00I9M8',
'043LG7223D0UB234B358EQ2F3O00IQPV',
'043LG7223D0UB234B358EQ2F3O00J4GB',
'043LG7223D0UB234B358EQ2F3O00JB00',
'043LG7223D0UB234B358EQ2F3O00K4KJ',
'043LG7223D0UB234B358EQ2F3O00K7IU',
'043LG7223D0UB234B358EQ2F3O00KL41',
'043LG7223D0UB234B358EQ2F3O00KPP7',
'043LG7223D0UB234B358EQ2F3O00L49T',
'043LG7223D0UB234B358EQ2F3O00LHEL',
'043LG7223D0UB234B358EQ2F3O00M1I1',
'043LG7223D0UB234B358EQ2F3O00MA66',
'043LG7223D0UB234B358EQ2F3O00MFLS',
'043LG7223D0UB234B358EQ2F3O00NBA5',
'043LG7223D0UB234B358EQ2F3O00OTFB',
'043LG7223D0UB234B358EQ2F3O00PC2T',
'043LG7223D0UB234B358EQ2F3O00PD68',
'043LG7223D0UB234B358EQ2F3O00Q27O',
'KK5MPL8GG911R0E1AC5QUIOP7003JQOE',
'KK5MPL8GG911R0E1AC5QUIOP7003JSBK',
'KK5MPL8GG911R0E1AC5QUIOP7003K8CV',
'KK5MPL8GG911R0E1AC5QUIOP7003KCBG',
'KK5MPL8GG911R0E1AC5QUIOP7003KD6A']

In [35]:
A3

['043LG7223D0UB234B358EQ2F3O004VVR',
 '043LG7223D0UB234B358EQ2F3O0059MM',
 '043LG7223D0UB234B358EQ2F3O005CGV',
 '043LG7223D0UB234B358EQ2F3O005D2O',
 '043LG7223D0UB234B358EQ2F3O005MRO',
 '043LG7223D0UB234B358EQ2F3O005SP1',
 '043LG7223D0UB234B358EQ2F3O0069T6',
 '043LG7223D0UB234B358EQ2F3O006CG3',
 '043LG7223D0UB234B358EQ2F3O006IA9',
 '043LG7223D0UB234B358EQ2F3O006RLE',
 '043LG7223D0UB234B358EQ2F3O0075GQ',
 '043LG7223D0UB234B358EQ2F3O00802I',
 '043LG7223D0UB234B358EQ2F3O008868',
 '043LG7223D0UB234B358EQ2F3O008AK9',
 '043LG7223D0UB234B358EQ2F3O008DR7',
 '043LG7223D0UB234B358EQ2F3O008GKK',
 '043LG7223D0UB234B358EQ2F3O0099L2',
 '043LG7223D0UB234B358EQ2F3O00B5O5',
 '043LG7223D0UB234B358EQ2F3O00B65T',
 '043LG7223D0UB234B358EQ2F3O00B6AE',
 '043LG7223D0UB234B358EQ2F3O00BH19',
 '043LG7223D0UB234B358EQ2F3O00BHB8',
 '043LG7223D0UB234B358EQ2F3O00BIIA',
 '043LG7223D0UB234B358EQ2F3O00BV5R',
 '043LG7223D0UB234B358EQ2F3O00BVPC',
 '043LG7223D0UB234B358EQ2F3O00CC04',
 '043LG7223D0UB234B358EQ2F3O00CMD0',
 

In [37]:
A4 = [
'KK5MPL8GG911R0E1AC5QUIOP7003KDVG',
'KK5MPL8GG911R0E1AC5QUIOP7003KIK8',
'KK5MPL8GG911R0E1AC5QUIOP7003KK8D',
'KK5MPL8GG911R0E1AC5QUIOP7003KUNP',
'KK5MPL8GG911R0E1AC5QUIOP7003L0UG',
'KK5MPL8GG911R0E1AC5QUIOP7003L1DK',
'KK5MPL8GG911R0E1AC5QUIOP7003L496',
'KK5MPL8GG911R0E1AC5QUIOP7003LBGF',
'KK5MPL8GG911R0E1AC5QUIOP7003LOU5',
'KK5MPL8GG911R0E1AC5QUIOP7003LUC9',
'KK5MPL8GG911R0E1AC5QUIOP7003OCKB',
'KK5MPL8GG911R0E1AC5QUIOP7003Q1A3',
'KK5MPL8GG911R0E1AC5QUIOP7003Q32C',
'KK5MPL8GG911R0E1AC5QUIOP7003QENV',
'KK5MPL8GG911R0E1AC5QUIOP7003QR6S',
'KK5MPL8GG911R0E1AC5QUIOP7003RBSL',
'KK5MPL8GG911R0E1AC5QUIOP7003RFLD',
'KK5MPL8GG911R0E1AC5QUIOP7003RIGA',
'KK5MPL8GG911R0E1AC5QUIOP7003RPQJ',
'KK5MPL8GG911R0E1AC5QUIOP7003RPT6',
'KK5MPL8GG911R0E1AC5QUIOP7003RS93',
'KK5MPL8GG911R0E1AC5QUIOP7003RTTM',
'KK5MPL8GG911R0E1AC5QUIOP7003RVSF',
'KK5MPL8GG911R0E1AC5QUIOP7003S7N9',
'KK5MPL8GG911R0E1AC5QUIOP7003SBSO',
'KK5MPL8GG911R0E1AC5QUIOP7003T6MG',
'KK5MPL8GG911R0E1AC5QUIOP7003T8EL',
'KK5MPL8GG911R0E1AC5QUIOP7003TE0M',
'KK5MPL8GG911R0E1AC5QUIOP7003TGO2',
'KK5MPL8GG911R0E1AC5QUIOP7003TKID',
'KK5MPL8GG911R0E1AC5QUIOP7003TM44',
'KK5MPL8GG911R0E1AC5QUIOP7003TN3A',
'KK5MPL8GG911R0E1AC5QUIOP7003TP6E',
'KK5MPL8GG911R0E1AC5QUIOP7003TT2M',
'KK5MPL8GG911R0E1AC5QUIOP7003TTL0',
'KK5MPL8GG911R0E1AC5QUIOP70040FR3',
'KK5MPL8GG911R0E1AC5QUIOP70041AG3',
'KK5MPL8GG911R0E1AC5QUIOP70044UV1',
'KK5MPL8GG911R0E1AC5QUIOP700461EV',
'KK5MPL8GG911R0E1AC5QUIOP70046HPQ',
'KK5MPL8GG911R0E1AC5QUIOP70046IN2',
'KK5MPL8GG911R0E1AC5QUIOP7004FQP9',
'KK5MPL8GG911R0E1AC5QUIOP7004IHRH',
'KK5MPL8GG911R0E1AC5QUIOP7004J48N',
'KK5MPL8GG911R0E1AC5QUIOP7004J8CA']

In [38]:
A4

['KK5MPL8GG911R0E1AC5QUIOP7003KDVG',
 'KK5MPL8GG911R0E1AC5QUIOP7003KIK8',
 'KK5MPL8GG911R0E1AC5QUIOP7003KK8D',
 'KK5MPL8GG911R0E1AC5QUIOP7003KUNP',
 'KK5MPL8GG911R0E1AC5QUIOP7003L0UG',
 'KK5MPL8GG911R0E1AC5QUIOP7003L1DK',
 'KK5MPL8GG911R0E1AC5QUIOP7003L496',
 'KK5MPL8GG911R0E1AC5QUIOP7003LBGF',
 'KK5MPL8GG911R0E1AC5QUIOP7003LOU5',
 'KK5MPL8GG911R0E1AC5QUIOP7003LUC9',
 'KK5MPL8GG911R0E1AC5QUIOP7003OCKB',
 'KK5MPL8GG911R0E1AC5QUIOP7003Q1A3',
 'KK5MPL8GG911R0E1AC5QUIOP7003Q32C',
 'KK5MPL8GG911R0E1AC5QUIOP7003QENV',
 'KK5MPL8GG911R0E1AC5QUIOP7003QR6S',
 'KK5MPL8GG911R0E1AC5QUIOP7003RBSL',
 'KK5MPL8GG911R0E1AC5QUIOP7003RFLD',
 'KK5MPL8GG911R0E1AC5QUIOP7003RIGA',
 'KK5MPL8GG911R0E1AC5QUIOP7003RPQJ',
 'KK5MPL8GG911R0E1AC5QUIOP7003RPT6',
 'KK5MPL8GG911R0E1AC5QUIOP7003RS93',
 'KK5MPL8GG911R0E1AC5QUIOP7003RTTM',
 'KK5MPL8GG911R0E1AC5QUIOP7003RVSF',
 'KK5MPL8GG911R0E1AC5QUIOP7003S7N9',
 'KK5MPL8GG911R0E1AC5QUIOP7003SBSO',
 'KK5MPL8GG911R0E1AC5QUIOP7003T6MG',
 'KK5MPL8GG911R0E1AC5QUIOP7003T8EL',
 

In [39]:
A = A1+A2+A3+A4

In [51]:
len(A)

250

In [52]:
file_path = "aflac_actorchat_5246.pkl"

# Read the pickled file and store its contents in the variable 'data_raw'
with open(file_path, 'rb') as file:
    data_raw = pickle.load(file)

In [53]:
data_raw

,session_id,start_date,end_date,call_type,agent_id,agent_name,timestamp,chat_text,entity_type,sentiment_score,...,num_transfers,overlap_time,max_silence,sentiment,num_holds,last_speaker,aperture,corpus 1,corpus 2,end
0,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,2.1875,"This greenthis is a sontaran, the application line called so recorded and monitored for quality assurance purposes.",agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
1,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,10.3750,May leave your first and your last name.,agent,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
2,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,11.5625,,client,0.000000,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
3,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,14.0625,My name is Beth and I was wondering if you could tell me what kind of um.,client,-0.089812,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
4,043LG7223D0UB234B358EQ2F3O000CF2,2022-10-19T15:23:07.000Z,2022-10-19T15:32:34.000Z,Inbound,20571029,leaha.santorum@sutherlandglobal.com,19.0000,What kind of insurance do you? What what you get? What you do? What's your?,client,-0.523072,...,NaN,53.6875,7.1875,NEUTRAL,1.0,client,-0.328834,-0.498714,-0.147273,0.212184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096705,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.0625,"Your welcome. I hope I was able to help out with information. You have a great rest of your day, Gustavo, and I'll reach out next week just to follow up, Okay?",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096706,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,583.1875,The.,client,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096707,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,591.1875,"Alright, sounds good. you have a good one, too.",client,0.956295,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827
1096708,SRC7K28UCT42HA3ALE47AJJ58400TI1B,2023-01-04T01:36:31.000Z,2023-01-04T01:46:30.000Z,Outbound,20571096,robert.hanmer@sutherlandglobal.com,594.0625,"Thank you take care. bye, bye.",agent,0.000000,...,NaN,29.4375,33.1250,NEUTRAL,0.0,client,-0.137542,-0.041391,-0.017339,0.764827


In [54]:
import pandas as pd

# Sample data


# Function to extract agent name
def get_agent_name(email):
    parts = email.split('@')[0].split('.')
    if len(parts) >= 2:
        return parts[0] + ' ' + parts[-1]
    else:
        return email

# Create a DataFrame with session_id and corresponding agent_name
result = []
for session_id in A:
    agent_email = data_raw[data_raw['session_id'] == session_id]['agent_email'].iloc[0]
    agent_name = get_agent_name(agent_email)
    result.append({'session_id': session_id, 'agent_name': agent_name})

df = pd.DataFrame(result)

print(df)

                           session_id          agent_name
0    L17LEQKG6H2A77JNOPQIFJ6ST40098D1       kevin petroff
1    043LG7223D0UB234B358EQ2F3O00BH0J          misha hill
2    KK5MPL8GG911R0E1AC5QUIOP7003D4TP      william shuler
3    043LG7223D0UB234B358EQ2F3O00EB9I      william shuler
4    L17LEQKG6H2A77JNOPQIFJ6ST400B9P0       jamerick boyd
5    L17LEQKG6H2A77JNOPQIFJ6ST40025P8      deborah singer
6    KK5MPL8GG911R0E1AC5QUIOP7002BA5D         dawn dawsey
7    7JCCO8JK0T7UH4H1Q3FILGCK2S002STA          misha hill
8    KK5MPL8GG911R0E1AC5QUIOP7004JH8Q       kevin petroff
9    SRC7K28UCT42HA3ALE47AJJ58400284N       kevin petroff
10   KK5MPL8GG911R0E1AC5QUIOP70028KQ5          lisa combs
11   043LG7223D0UB234B358EQ2F3O000JGV         dawn dawsey
12   LKL9VL2RUP3SB6QFMUHC830T7000E5K9          lisa combs
13   KK5MPL8GG911R0E1AC5QUIOP7002M4O1       kevin petroff
14   KK5MPL8GG911R0E1AC5QUIOP7005UF7E       jamerick boyd
15   043LG7223D0UB234B358EQ2F3O00JB2I       nikkia laster
16   KK5MPL8GG

In [56]:
df.to_excel(cwd + "//fffffffffff.xlsx")